In [1]:
from domino import DominoSlicer
import numpy as np
import sklearn
sklearn.__version__

'1.3.2'

In [2]:
places8_clip_emd = "places8_image_features_clip.npy"
places8_image_preds = "places8_image_preds_1hot.npy"
places8_targets = "places8_image_targets.npy"
clip_emb = np.load(places8_clip_emd)
image_1hot_preds = np.load(places8_image_preds)
places8_targets = np.load(places8_targets)

# indexes_1 = np.where(places8_targets==1)
# clip_emb_1 = clip_emb[indexes_1]
# image_1hot_preds_1 = image_1hot_preds[indexes_1]
# places8_targets_1 = places8_targets[indexes_1]
# print("Only one class:")
# print(f"Embed shape {clip_emb_1.shape} | target shape {places8_targets_1.shape} | preds 1hot encode {image_1hot_preds_1.shape}")
print("Data with all 8 classes:")
print(f"Embed shape {clip_emb.shape} | target shape {places8_targets.shape} | preds one-hot encode shapes {image_1hot_preds.shape}")

Data with all 8 classes:
Embed shape (364806, 512) | target shape (364806,) | preds one-hot encode shapes (364806, 8)


In [3]:
for i in image_1hot_preds:
    if np.count_nonzero(i) != 1:
        print("erro")

In [4]:
np.unique(places8_targets, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 51655, 100012,  41849,  33763,  21889,  89458,  12633,  13547]))

In [5]:
image_1hot_preds= image_1hot_preds.astype('int8')
image_1hot_preds.dtype, image_1hot_preds.shape

(dtype('int8'), (364806, 8))

In [6]:
slicer = DominoSlicer(
    y_log_likelihood_weight=40,
    y_hat_log_likelihood_weight=40,
    n_mixture_components=65,
    n_slices=5)

In [7]:
slicer.get_params()

{'n_slices': 5,
 'covariance_type': 'diag',
 'n_pca_components': 128,
 'n_mixture_components': 65,
 'init_params': 'kmeans',
 'confusion_noise': 0.001,
 'y_log_likelihood_weight': 40,
 'y_hat_log_likelihood_weight': 40,
 'max_iter': 100,
 'random_state': None}

In [8]:
_ = slicer.fit(embeddings = clip_emb, targets = places8_targets, pred_probs = image_1hot_preds)

 28%|████████████████████████████████████████████████▋                                                                                                                             | 28/100 [01:38<04:14,  3.53s/it]


In [9]:
predict = slicer.predict(embeddings= clip_emb, targets = places8_targets, pred_probs = image_1hot_preds)

In [10]:
predict

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)

In [11]:
np.unique(np.argmax(predict, axis=-1), return_counts=True)

(array([0, 1, 2, 3, 4]), array([349132,   7327,   4516,    595,   3236]))

In [12]:
np.unique(places8_targets, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 51655, 100012,  41849,  33763,  21889,  89458,  12633,  13547]))

In [13]:
#executar por classe

In [14]:
image_1hot_as_class_idxs = np.argmax(image_1hot_preds, axis=1)
image_1hot_as_class_idxs

array([0, 0, 0, ..., 1, 1, 1])

In [15]:
np.unique(image_1hot_as_class_idxs, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 53052, 110713,  48566,  24107,  16269,  79675,  17798,  14626]))

In [17]:
slicer2 = DominoSlicer(
    y_log_likelihood_weight=40,
    y_hat_log_likelihood_weight=40,
    n_mixture_components=65,
    n_slices=5)

In [18]:
_ = slicer2.fit(embeddings = clip_emb,
                targets = places8_targets,
                pred_probs = image_1hot_as_class_idxs)

 32%|███████████████████████████████████████████████████████▋                                                                                                                      | 32/100 [01:57<04:09,  3.66s/it]


In [19]:
predict2 = slicer2.predict(embeddings = clip_emb,
                           targets = places8_targets,
                           pred_probs = image_1hot_as_class_idxs)

In [20]:
predict2

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)

In [21]:
np.unique(np.argmax(predict2, axis=-1), return_counts=True)

(array([0, 1, 2, 3, 4]), array([351865,   1986,    686,   8220,   2049]))